# 01_EDA v0.1: Análisis Exploratorio de Datos (Robusto)

Este notebook realiza un análisis exploratorio exhaustivo de los datos disponibles para el proyecto de detección de fraude financiero. 

**Objetivos de esta versión 0.1**:
1. Cargar y revisar todos los archivos de `data/raw/` (o `data/backup/` si los ligeros no existen en raw).  
2. Normalizar tipos (fechas, montos, flags “Yes/No” → 1/0) y revisar datos faltantes.  
3. Combinar transacciones con etiquetas de forma correcta (según la estructura real de `train_fraud_labels.json`).  
4. Detectar duplicados, outliers y valores inconsistentes.  
5. Explorar:
   - Distribución de montos (`amount`).  
   - Balance de la variable objetivo (`target`).  
   - Fraude por estado y ciudad.  
   - Fraude por hora del día y día de la semana.  
   - Fraude por tipo de tarjeta.  
   - Top MCCs donde hay más fraudes.  
6. Calcular algunas correlaciones básicas y visualizar mapa de calor.  
7. Hacer un pequeño preprocesamiento (generar columna `hour`, flags binarias, parsear fechas).  
8. Guardar el dataset combinado limpio en `data/processed/` para la fase de modelado.

In [25]:
# 2.1 Importar librerías esenciales
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

# 2.2 Estilo para gráficos
%matplotlib inline
plt.style.use('default')

# 2.3 Opciones de pandas para visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', '{:.2f}'.format)

# 2.4 Función auxiliar para inspeccionar cualquier DataFrame
def inspeccionar_df(df, nombre):
    """
    Imprime info básica: shape, dtypes, nulos y duplicados de un DataFrame.
    """
    print(f"\n--- Inspección de {nombre} ---")
    print("Dimensiones:", df.shape)
    print("\nTipos de columnas:\n", df.dtypes)
    print("\nValores faltantes por columna:\n", df.isnull().sum().sort_values(ascending=False))
    print(f"\nDuplicados exactos (filas idénticas): {df.duplicated().sum()}")
    if "transaction_id" in df.columns:
        print("Duplicados en 'transaction_id':", df["transaction_id"].duplicated().sum())
    print("-" * 60 + "\n")

# 2.5 Semilla para reproducibilidad (opcional)
import random
random.seed(42)


In [29]:
path_labels = "../data/backup/train_fraud_labels.json"
print("¿Existe el archivo?", os.path.exists(path_labels))


# 3.2.2 Detectar tipo de estructura: JSONL, lista o dict
with open(path_labels, "r") as f:
    first_line = f.readline().strip()
    f.seek(0)  # Volver al inicio del archivo

    try:
        json.loads(first_line)
        is_json_lines = True
    except json.JSONDecodeError:
        is_json_lines = False


# 3.2.3 Cargar el archivo según la estructura detectada
try:
    if is_json_lines:
        df_labels = pd.read_json(path_labels, lines=True)
        print("✅ 3.2.3 Cargado como JSON Lines.")
    else:
        with open(path_labels, "r") as f:
            raw_data = json.load(f)

        if isinstance(raw_data, list):
            df_labels = pd.DataFrame(raw_data)
            print("✅ 3.2.3 Cargado como lista de diccionarios.")
        elif isinstance(raw_data, dict):
            df_labels = pd.DataFrame.from_dict(raw_data, orient="index").reset_index()
            df_labels.columns = ["transaction_id", "target"]  # Ajustar si es necesario
            print("✅ 3.2.3 Cargado como diccionario con IDs.")
        else:
            raise ValueError("❌ 3.2.3 Estructura JSON no reconocida.")

    print("Dimensiones de df_labels:", df_labels.shape)
    display(df_labels.head())

except Exception as e:
    print("❌ 3.2.3 Error durante la carga:", e)

¿Existe el archivo? True
✅ 3.2.3 Cargado como JSON Lines.
Dimensiones de df_labels: (1, 1)


,target
0,"{'10649266': 'No', '23410063': 'No', '9316588'..."


In [36]:
# 3.3.1 Cargar transactions_data.csv
path_transactions = "../data/backup/transactions_data.csv"
df_transacciones = pd.read_csv(path_transactions)
inspeccionar_df(df_transacciones, "df_transacciones")

# 3.3.2 Cargar train_fraud_labels.json 'bruto'
path_labels = "../data/backup/train_fraud_labels.json"
with open(path_labels, "r") as f:
    raw_labels = json.load(f)

# 3.3.3 Analizar estructura de raw_labels antes de convertirlo
print("Tipo de raw_labels:", type(raw_labels))
if isinstance(raw_labels, dict):
    print("Claves del diccionario raíz:", raw_labels.keys())
elif isinstance(raw_labels, list):
    print("Primeros 5 elementos de la lista raw_labels:")
    display(raw_labels[:5])
else:
    print("Estructura inesperada en raw_labels.")



--- Inspección de df_transacciones ---
Dimensiones: (13305915, 12)

Tipos de columnas:
 id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object

Valores faltantes por columna:
 errors            13094522
zip                1652706
merchant_state     1563700
id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
mcc                      0
dtype: int64

Duplicados exactos (filas idénticas): 0
------------------------------------------------------------

Tipo de raw_labels: <class 'dict'>
Claves del diccionario raíz: dict_keys(['target'])


In [31]:
with open(path_labels, "r") as f:
    raw_labels = json.load(f)
type(raw_labels)


dict

In [37]:
# 4.3 Caso C: targets sin IDs, se asignan por orden
target_list = raw_labels["target"]

if isinstance(target_list, list) and len(target_list) < df_transacciones.shape[0]:
    n = len(target_list)
    print(f"✅ raw_labels corresponde a las primeras {n} filas de df_transacciones")

    # Dividir en conjunto etiquetado y no etiquetado
    df_train = df_transacciones.iloc[:n].copy()
    df_train["target"] = target_list

    df_test = df_transacciones.iloc[n:].copy()
    df_test["target"] = np.nan

    # Concatenar ambos conjuntos
    df_full = pd.concat([df_train, df_test], ignore_index=True)
    print("✅ df_full creado exitosamente")
    print("Dimensiones de df_full:", df_full.shape)

    # Validación
    print("Cantidad de targets 0:", (df_full["target"] == 0).sum())
    print("Cantidad de targets 1:", (df_full["target"] == 1).sum())
    print("Cantidad de targets NaN:", df_full["target"].isna().sum())

    display(df_full.head())
else:
    print("❌ Estructura de raw_labels no corresponde a la esperada en el Caso C.")


❌ Estructura de raw_labels no corresponde a la esperada en el Caso C.
